In [ ]:
import open3d as o3d
import os
import numpy as np
from tqdm.notebook import tqdm
import math
import json
import random
import uuid
import ifcopenshell
from ifcopenshell import template
import ifcopenshell.geom

from src.visualisation import *
from src.ifc import *
from src.elements import create_pipe, create_elbow
from src.dataset import *
from src.preparation import *
from src.cloud import add_noise

# create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
from numpy.random import default_rng

In [6]:
random.seed(10)
rng = default_rng()

### CLOI Dataset Creation

The following section converts CLOI scans into a pcd dataset.

In [ ]:
# combine clouds
data_path = "/mnt/f/datasets/export/export/"
max_points = 4096

In [ ]:
classes = os.listdir(data_path)
print(classes)

In [ ]:
all_classes = []
element_count = 0
error_count = 0
for i, cl in enumerate(classes):
    all_elements = []
    class_path = data_path + cl
    elements = os.listdir(class_path)
    for j, el in tqdm(enumerate(elements)):
        try:
            element = np.loadtxt(class_path + "/" + el)

            # downsample
            if len(element) > 0 and element.ndim == 2 and element.shape[1] == 4:
                if len(element) > max_points:
                    # idx = np.random.randint(element.shape[0], size=max_points)
                    # element = element[idx :]
                    element = np.random.permutation(element)[:max_points]

                element = np.delete(element, 3, axis=1)  # remove point index
                element = np.insert(
                    element, 3, values=[element_count], axis=1
                )  # add element index
                # print(element.shape)
                element_count += 1
                all_elements.append(element)
        except Exception as E:
            error_count += 1

    all_elements = np.vstack(all_elements)
    all_elements = np.insert(all_elements, 4, values=[i], axis=1)  # add class index
    all_classes.append(all_elements)

all_classes = np.concatenate(all_classes)
print(all_classes.shape)
print("errors: ", error_count)

# print(points[0])

In [ ]:
pcd = o3d.t.geometry.PointCloud()
points = all_classes[:, 0:3]
el_index = [[i] for i in all_classes[:, 3]]
cl_index = [[i] for i in all_classes[:, 4]]


pcd.point["positions"] = o3d.core.Tensor(points)
pcd.point["elements"] = o3d.core.Tensor(el_index)
pcd.point["classes"] = o3d.core.Tensor(cl_index)

o3d.t.io.write_point_cloud("water2.pcd", pcd)

## Pipe parameter detection

### Generation of synethetic IFC element dataset

#### Dataset creation process
1. Generate params for element model
2. Generate ifc models
3. Convert to obj models using ifcConvert (ifc2obj.py script (synthetic) OR element_to_obj function (BP)) 
4. Convert to partially occluded EXR images using render_depth.py script *./blender -b -P render_depth.py ../industrial-facility-relationships/output/obj/ output/*
5. Convert to point clouds using process_exr.py script *python process_exr.py output/exr/ output/intrinsics.txt output/*
6. Combine multiple views of object to create training and testing datasets (for this, the metadata generated in step 3 must be pasted to individual metadata files for each class. This steps outputs a metadata_new file. set multiple=False for BP datasets)

BP datasets are created by following steps 3->6 above after splitting extracted IFC into multiple IFCs (to fix overlapping pieces)

##### Step 1 & 2. IFC model generation


In [4]:
density = 2048
sample_size = 4096
config_path = "config/pipeline.json"
pcd_path = "/home/haritha/documents/blender-2.79-linux-glibc219-x86_64/output_elbow/pcd/"
blueprint = "data/sample.ifc"
num_scans = 16

**Elbow - modelled as an IfcRevolvedAreaSolid model**

*params:*

- position - 3D coordinate
- direction - 3D vector, axis of extrusion (normal to axis of revolution) (z>=0)
- axis_position - 2D coordinate, relative to position
- angle - angle of revolution (0 -> pi)
- radius


**Pipe - modelled as an IfcExtrudedAreaSolid model**

*params:*
    
- position - 3D coordinate
- extrusion_direction - 3D vector (z>=0)
- length
- radius


**Tee - modelled as a combination of 2 IfcRevolvedAreaSolid models**

The two pipes are each susbtracted from the other to create an IfcCsgSolid using an IfcBooleanResult.

*params:*
    
- position - 3D coordinate
- extrusion_direction1 - 3D vector (z>=0)
- extrusion_direction2 - 3D vector (z>=0)
- length1
- length2 - percentage of length1
- tee angle - 90 degrees or within an angle range
- radius1
- radius2 - same as radius1 or percentage of radius1


In [ ]:
# synthetic_dataset(config_path, sample_size, "pipe", 'occluded', blueprint, 0)

*Use external scripts to convert above IFC dataset into ocluded PCD dataset. (step 3, 4 & 5)*

##### Step 6. Test / train dataset creation

1. merge multiple views
2. sample to standard density
3. generate training and testing dataset



In [ ]:
create_merged_dataset(
    pcd_path, "completion/", "elbow", num_scans, density, 3, 0.0, False, multiple=True
)

In [ ]:
# create occluded dataset with non occluded ground truth for point cloud completion
create_completion_dataset(
    pcd_path, "completion_noisy/", "elbow", num_scans, density, 1, 0.1, False, multiple=False, noise=True
)

In [ ]:
print(7 * sum([i for i in range(1, 5 + 1)]))

In [ ]:
# add noise  to existing dataset
input_dir = "output/tee/train/"
output_dir = "output/noisy/tee/train/"
noise_size = 128
cloud = o3d.geometry.PointCloud()

files = os.listdir(input_dir)

for f in tqdm(files):
    points = np.array(o3d.io.read_point_cloud(input_dir + f).points)
    noisy_points = add_noise(points, noise_size, rng)
    noisy_points = o3d.utility.Vector3dVector(noisy_points)
    cloud.points = noisy_points
    o3d.io.write_point_cloud(output_dir + f, cloud)

In [ ]:
# script to recover pipe metadata from metadata_new since the metadata file has magically gotten corrupted

new_m = "output/pipe/metadata_new.json"
f = open(new_m, "r")
metadata = json.load(f)
meta_dict = {}

for m in metadata:
    meta_dict[metadata[m]["initial_ifc"]] = {
        "radius": metadata[m]["radius"],
        "direction": metadata[m]["direction"],
        "length": metadata[m]["length"],
        "position": metadata[m]["position"],
    }

In [ ]:
print(meta_dict.keys())

In [ ]:
import pandas as pd

In [ ]:
file_path = "responses.csv"
data = pd.read_csv(file_path, sep=",")
# Display the first few rows of the dataframe
print(data.head())

In [ ]:
no_of_terms = data["Number of terms in postgrad accommodation"]

In [ ]:
print(category_based_on_terms)

In [ ]:
import pandas as pd

# Assuming the variable 'data' contains the DataFrame
data["category"] = pd.cut(
    data["Number of terms in postgrad accommodation"],
    bins=[-1, 0, 3, 6, float("inf")],
    labels=[1, 2, 3, 4],
)

In [ ]:
print(data["category"])

In [ ]:
data.to_csv("data.csv", index=False)

In [ ]:
import pandas as pd

# Assuming the DataFrame is read from a CSV
file_path = "data.csv"
data = pd.read_csv(file_path)

# Select columns "1st choice" to "10th choice"
columns_of_interest = [f"{i}th choice" for i in range(1, 11)]
columns_of_interest[0] = "1st choice"  # Correct the first item
columns_of_interest[1] = "2nd choice"  # Correct the second item
columns_of_interest[2] = "3rd choice"  # Correct the third item

# Extract the specific columns
choices_data = data[columns_of_interest]

# Define the prefixes to check
prefixes = ("OX", "OR", "PS", "HR")


# Function to check if the value starts with any of the specified prefixes
def starts_with_prefix(value):
    if pd.isna(value):
        return False
    return value.startswith(prefixes)


# Apply the function to each cell in the DataFrame
matches_prefix = choices_data.applymap(starts_with_prefix)

# Sum the True values across the row to check if there are at least four
data["meets_criteria"] = matches_prefix.sum(axis=1) >= 4

# Check if all 10 values in the row are unique
data["meets_uniqueness_criteria"] = choices_data.apply(
    lambda row: len(set(row)) == 10, axis=1
)


# Display the updated DataFrame with the new 'meets_criteria' column
print(data[["meets_criteria"] + columns_of_interest])

In [ ]:
data.to_csv("data_hostels_unique.csv", index=False)